# File operations in Python

In [ ]:
import tempfile
with tempfile.TemporaryFile("w+b") as mytempfile:
  mytempfile.write(b"Hello world!")
  mytempfile.seek(1)
  bla = mytempfile.read().decode("utf-8")


In [ ]:
import os
import s3fs
import zipfile
import tempfile
import numpy as np
from tensorflow import keras
from pathlib import Path
import logging


AWS_ACCESS_KEY="aws_access_key"
AWS_SECRET_KEY="aws_secret_key"
BUCKET_NAME="bucket_name"


def get_s3fs():
  return s3fs.S3FileSystem(key=AWS_ACCESS_KEY, secret=AWS_SECRET_KEY)


def zipdir(path, ziph):
  # Zipfile hook to zip up model folders
  length = len(path) # Doing this to get rid of parent folders
  for root, dirs, files in os.walk(path):
    folder = root[length:] # We don't need parent folders! Why in the world does zipfile zip the whole tree??
    for file in files:
      ziph.write(os.path.join(root, file), os.path.join(folder, file))

            
def s3_save_keras_model(model, model_name):
  with tempfile.TemporaryDirectory() as tempdir:
    model.save(f"{tempdir}/{model_name}")
    # Zip it up first
    zipf = zipfile.ZipFile(f"{tempdir}/{model_name}.zip", "w", zipfile.ZIP_STORED)
    zipdir(f"{tempdir}/{model_name}", zipf)
    zipf.close()
    s3fs = get_s3fs()
    s3fs.put(f"{tempdir}/{model_name}.zip", f"{BUCKET_NAME}/{model_name}.zip")
    logging.info(f"Saved zipped model at path s3://{BUCKET_NAME}/{model_name}.zip")
 

def s3_get_keras_model(model_name: str) -> keras.Model:
  with tempfile.TemporaryDirectory() as tempdir:
    s3fs = get_s3fs()
    # Fetch and save the zip file to the temporary directory
    s3fs.get(f"{BUCKET_NAME}/{model_name}.zip", f"{tempdir}/{model_name}.zip")
    # Extract the model zip file within the temporary directory
    with zipfile.ZipFile(f"{tempdir}/{model_name}.zip") as zip_ref:
        zip_ref.extractall(f"{tempdir}/{model_name}")
    # Load the keras model from the temporary directory
    return keras.models.load_model(f"{tempdir}/{model_name}")
  


inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="mean_squared_error")

# Save the model to S3
s3_save_keras_model(model, "my_model") 

# Load the model from S3
loaded_model = s3_get_keras_model("my_model")

In [ ]:
import os

import tempfile
from zipfile import ZipFile

from shutil import copyfile

with tempfile.TemporaryDirectory() as tempdir:
    model_name = "testmodel"
    source_path = f"{tempdir}/{model_name}"
    if not os.path.exists(source_path):
        try:
            os.makedirs(source_path)
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise         
    with open(f"{tempdir}/{model_name}/myfile.txt", mode = 'w', encoding = 'utf-8') as f:
        f.write("my first file\n")
        f.write("another line\n")
        f.write("third line\n")
    # create a ZipFile object
    with ZipFile(f"{source_path}.zip", 'w') as zipObj:
        # Iterate over all the files in directory
        for folderName, subfolders, filenames in os.walk(source_path):
            for filename in filenames:
                #create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath, os.path.basename(filePath))
    input()
    copyfile(f"{tempdir}/{model_name}.zip", "/home/markus/testing.zip")